## [Link para documento overleaf](https://www.overleaf.com/project/677fe7fd4bf7cc5092992e57) 

In [0]:
#Bibliotecas
import pandas as pd
from typing import List,Callable


In [0]:
#Funções Intermédiarias da Função Principal
def query(arquivo:str) -> pd.DataFrame:
    caminho_base = "/Workspace/Users/joaopedroalexandrino6@hotmail.com/desafio_kinea_2025/ScriptsSQL"
    caminho = caminho_base +'/' + arquivo + '.sql'
    with open(caminho, 'r') as f:
        query = f.read()
    df = spark.sql(query).toPandas()
    return df


def extrair_informacoes() -> List[pd.DataFrame]:
    '''
    Objetivo:
        Extrair informações necessárias para o Cálculo da Régua
    Inputs:
        None
    Output:
        inf_fundos: DataFrame com Patrimônio Líquido, Range Mínimo e Máximo de Crédito para cada fundo ['fundo','PL','l_min','l_max','v_min','v_max']
        peso_book_fundos: DataFrame com Percentual de Alocação de cada Book (Esse perccentual é referente a parte disponivel para crédito) para cada fundo ['fundo','book_macro','peso_book_macro']
        credito_aportado: DataFrame com valor de crédito alocado em cada book em cada fundo ['fundo','book','produto',alocado']
        restricoes_book_micro: DataFrame com as restrições book micro [book','fundo','flag']
        restricoes_fundo_restrito: Dataframe com restrições fundo restrito ['ativo',fundo','flag']
        ativos_books: DataFrame com ativos e seus books ['ativo','book_micro'] (Pode ser que não tenhamos todos cadastrados)
    '''
    #Tabela Base
    tabela_base = query("verificacao_range_e_book")
    tabela_base_aportado = query("credito_aportado")


    #Filtrando inf_fundos ['fundo','PL','l_min','l_max','v_min','v_max']
    inf_fundos = tabela_base[['fundo','PL','alocacao_min_porcen','alocacao_max_porcen','(alocacao_minima * PL)','(alocacao_maxima * PL)']].drop_duplicates('fundo').rename(columns={'(alocacao_minima * PL)':'v_min','(alocacao_maxima * PL)':'v_max','alocacao_min_porcen':'l_min','alocacao_max_porcen':'l_max'})
    
    #Filtrando peso_book_fundos ['fundo','book_macro','peso']
    peso_book_fundos = tabela_base[['fundo','peso','book_macro']].drop_duplicates(['fundo','book_macro'])

    #Filtrando Crédito Aportado ['fundo','ativo',alocado']
    credito_aportado = tabela_base_aportado.rename(columns={'TradingDesk':'fundo','Product':'ativo','Position':'alocado'}).drop(columns=["PositionDate"])

    #Filtrando restricoes_book_micro ['fundo','book_micro','flag']
    restricoes_book_micro = tabela_base[['fundo','book_micro','flag']].drop_duplicates(['fundo','book_micro'])

    #Filtrando restricoes_fundo_restrito ['ativo','fundo_restrito']
    restricoes_fundo_restrito = tabela_base[['ativo','fundo_restrito']].drop_duplicates(['ativo']) #Procurar Entender por que nessa query tem 303 e direto na tabela 305

    #Filtrando ativos_books ['ativo',book_micro']
    ativos_books = tabela_base[['ativo','book_micro']].drop_duplicates(['ativo'])

    return [inf_fundos,peso_book_fundos,credito_aportado,restricoes_book_micro,restricoes_fundo_restrito,ativos_books]

def calculo_regua_macro(inf_fundos: pd.DataFrame, peso_books_fundos:pd.DataFrame,credito_total:float) -> pd.DataFrame:
    '''
    Objetivo:
        Cálculo da Régua
    Inputs:
        inf_fundos: DataFrame com Patrimônio Líquido, Range Mínimo e Máximo de Crédito para cada fundo ['fundo','PL','l_min','l_max']
        peso_books_fundos: DataFrame com Percentual de Alocação de cada Book (Esse perccentual é referente a parte disponivel para crédito) para cada fundo ['fundo','book','peso']
    Output:
        regua: DataFrame com as informações da régua que contém as alocações ideais ['fundo','book','percentual_alocacao']
    '''

    #Funções Auxiliares
    def percentual_alocacao_range_total(credito_total:float,inf_fundos: pd.DataFrame) -> float:
        return (credito_total - inf_fundos['v_min'].sum())/ (inf_fundos['v_max'].sum() - inf_fundos['v_min'].sum())
    
    def percentual_credito_relativo_pl(percentual_alocacao_range_total:float,inf_fundo: pd.Series) -> dict:
        return inf_fundo['l_min'] + percentual_alocacao_range_total * (inf_fundo['l_max'] - inf_fundo['l_min'])
    
    def percentual_book_relativo_pl(percentual_credito_relativo_pl:float,peso_book_fundo: float) -> float:
        return percentual_credito_relativo_pl * peso_book_fundo
    
    def valor_book_relativo_pl(percentual_book_relativo_pl:float,pl:float) -> float:
        return pl *percentual_book_relativo_pl

    #Cálculo da Régua
    percentual_alocacao_range_total = percentual_alocacao_range_total(credito_total,inf_fundos)
   
    percentuais_credito_relativo_pl = {}
    for index,row in inf_fundos.iterrows():
        percentuais_credito_relativo_pl[row['fundo']] = percentual_credito_relativo_pl(percentual_alocacao_range_total,row)

    percentuais_books_relativo_pl = pd.DataFrame(columns=['fundo','book','percentual'])
    for fundo,percentual_credito_relativo_pl in percentuais_credito_relativo_pl.items():
        for index,row in peso_books_fundos[peso_books_fundos['fundo'] == fundo].iterrows():
            print(row)
            percentuais_books_relativo_pl.loc[len(percentuais_books_relativo_pl)] = [fundo,row['book_macro'],percentual_book_relativo_pl(percentual_credito_relativo_pl,row['peso'])]

    
    valores_books_relativos_pl = pd.DataFrame(columns=['fundo','book','valor'])
    for index,row in percentuais_books_relativo_pl.iterrows():
        valores_books_relativos_pl.loc[len(valores_books_relativos_pl)] = [row['fundo'],row['book'],valor_book_relativo_pl(row['percentual'],inf_fundos[inf_fundos['fundo'] == row['fundo']]['PL'].values[0])]
   
    regua = pd.DataFrame(columns=['fundo','book','percentual_alocacao'])
    books = valores_books_relativos_pl['book'].unique()
    fundos = valores_books_relativos_pl['fundo'].unique()
    somas_books = {}
    for book in books:
        somas_books[book] = valores_books_relativos_pl[valores_books_relativos_pl['book'] == book]['valor'].sum()
    for fundo in fundos:
        for index,row in valores_books_relativos_pl[valores_books_relativos_pl['fundo'] == fundo].iterrows():
            regua.loc[len(regua)] = [fundo,row['book'],row['valor']/somas_books[row['book']]]

    return regua


In [0]:
#Função Principal
def solucao(checagens: List[Callable],ordens: pd.DataFrame) -> pd.DataFrame:
    '''
    Função principal que recebe como input:
    1. checagens: Lista de funções geradas pelo ChatGPT
    2. ordens: DataFrame ['Ticker','Amount','Price'] representando uma grupo de ordens a serem boletadas
    e retorna:
    1. ordens_final: DatFrame ['Ticker','Amount','Price','Fundo'] representando quanto irá para cada fundo
    '''
    #1.Etapa - Extrair Informações dos Fundos (Patrimônio Líquido, Range Mínimo e Máximo de Crédito, Pesos de cada Book em cada Fundo)
    dataframes = extrair_informacoes()
    inf_fundos = dataframes[0]
    peso_books = dataframes[1]
    credito_aportado = dataframes[2]
    
    #2.Etapa -> Calcular a Régua Book Macro 
    regua_macro = calculo_regua_macro(dataframes[0],dataframes[1],credito_aportado['alocado'].sum())

    #3 Etapa -> Aplicar Restrição de book micro e fundo restrito
   

   #4. etapa -> Passar por funções de verificação

   #4.1 -> Calcular quanto está errado e redistribuir
   #4.1.1 -> Precisamos de um método de otimização? Gradiente ou outrca coisa? Se sim Desenvolver teórioco
   #4.1.2 -> Avaliar Tempo de Execução com diferentes abordagens

   #5 -> Reverificar  e caso passe entregar o resultado final

teste = solucao([],pd.DataFrame())
teste.head(100)

fundo                          KAT
peso                           0.0
book_macro    Debenture_Fechada_HG
Name: 0, dtype: object
fundo                          KAT
peso                           0.0
book_macro    Debenture_Fechada_HY
Name: 11, dtype: object
fundo                    KAT
peso                     0.0
book_macro    Estruturado_HG
Name: 13, dtype: object
fundo                    KAT
peso                     0.0
book_macro    Estruturado_HY
Name: 23, dtype: object
fundo              KAT
peso          0.348123
book_macro          HG
Name: 39, dtype: object
fundo              KAT
peso          0.176598
book_macro          MY
Name: 40, dtype: object
fundo              KAT
peso          0.323779
book_macro          HY
Name: 58, dtype: object
fundo                          ID2
peso                      0.218743
book_macro    Debenture_Fechada_HG
Name: 302, dtype: object
fundo                          ID2
peso                      0.031335
book_macro    Debenture_Fechada_HY
Name: 3

,fundo,book,percentual_alocacao
0,KAT,Debenture_Fechada_HG,0.000000
1,KAT,Debenture_Fechada_HY,0.000000
2,KAT,Estruturado_HG,0.000000
3,KAT,Estruturado_HY,0.000000
4,KAT,HG,0.006204
...,...,...,...
95,RFA,HG,0.012897
96,RFA,MY,0.002553
97,RFA,HY,0.011866
98,KOP,Debenture_Fechada_HG,0.250431


In [0]:
teste[teste['book'] == 'Estruturado_HY']['percentual_alocacao'].sum()

1.0

In [0]:
#Teste Cálculo Régua - Exemplo feito pelo Ali na Reunião do dia 09/01/2025 (https://drive.google.com/file/d/1O-bVrF1At0xZNQIGmIOQ1Oi7YiSkGD0u/view?usp=sharing)
credito_total = 40
teste_inf_fundos = pd.DataFrame({
        'fundo':['RFA','APO','ID2'],
        'PL':[100,50,70],
        'l_min':[0.1,0.1,0.1],
        'l_max':[0.2,0.2,0.4]
})
teste_books_fundos  = pd.DataFrame({
        'fundo':['RFA','RFA','RFA','APO','APO','APO','ID2','ID2','ID2'],
        'book':['HG','MY','HY','HG','MY','HY','HG','MY','HY'],
        'peso_book':[0.53778,0.1634,0.29882,0.23006,0.433212,0.336729,0.41028,0.20813,0.38159]
})
resposta_nossa = calculo_regua(teste_inf_fundos,teste_books_fundos,credito_total)
resposta_ali = pd.DataFrame({
        'fundo':['RFA','RFA','RFA','APO','APO','APO','ID2','ID2','ID2'],
        'book':['HG','MY','HY','HG','MY','HY','HG','MY','HY'],
        'dist_regua':[0.4753,0.2623,0.3275,0.1017,0.3478,0.1845,0.4230,0.3899,0.4879]
        
})
print(resposta_nossa['percentual_alocacao'])
print(resposta_ali['dist_regua'])

0    0.475293
1    0.262353
2    0.327520
3    0.101664
4    0.347780
5    0.184535
6    0.423043
7    0.389866
8    0.487946
Name: percentual_alocacao, dtype: float64
0    0.4753
1    0.2623
2    0.3275
3    0.1017
4    0.3478
5    0.1845
6    0.4230
7    0.3899
8    0.4879
Name: dist_regua, dtype: float64


In [0]:
teste = extrair_informacoes()
teste.head()

,fundo,ativo,alocado
0,134,ACUPET7 7.5% 01/13/2032,4.618478e+07
1,134,AEGPA0,1.727806e+06
2,134,AEGPA3,3.286067e+07
3,134,AEGPA7,6.156705e+06
4,134,AEGPA8,2.247062e+06


In [0]:
#Teste Funções Imaginárias GPT
